In [93]:
#导入需要的模块
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [94]:
#数据读取
train_set_1 = pd.read_csv('train_hf_data.csv',header=0,index_col=0)
train_set_1_label = pd.read_csv('train_hf_label.csv',header=0,index_col=0)
test_set_1 = pd.read_csv('test_hf_data.csv',header=0,index_col=0)
test_set_1_label = pd.read_csv('test_hf_label.csv',header=0,index_col=0)

In [95]:
#设计函数清理不在给定范围内的样本
def data_clean(data):
    D = data
    D = D.drop(D[(D.age < 40) | (D.age > 95)].index)
    D = D.drop(D[(D.creatinine_phosphokinase < 23) | (D.creatinine_phosphokinase > 7861)].index)
    D = D.drop(D[(D.ejection_fraction < 14) | (D.ejection_fraction > 80)].index)
    D = D.drop(D[(D.platelets < 25010) | (D.platelets > 850000)].index)
    D = D.drop(D[(D.serum_creatinine < 0.5) | (D.serum_creatinine > 9.4)].index)
    D = D.drop(D[(D.serum_sodium < 114) | (D.serum_sodium > 148)].index)
    D = D.drop(D[(D.time < 4) | (D.time > 285)].index)
    return D.reset_index(drop=True)

In [96]:
#设计函数将数据按附件给的正常范围进行归一化处理
#这个函数是用的老师给的数据范围进行归一化的
def normalization(data):
    D = np.array(data)
    min_range = np.array([[40,55],[0,1],[23,7838],[0,1],[14,66],[0,1],[25010,825000],[0.5,8.9],[114,34],[0,1],[0,1],[4,281]])
    def maxminNormalize(X,X_min,X_range):
        return (X-X_min) / X_range
    for i in range(len(D[0]-1)):
        for j in range(len(D[:,0]-1)):
            D[j][i] = maxminNormalize(D[j][i],min_range[i][0],min_range[i][1])
    return D     

In [97]:
#对数据进行清洗
#对训练集清洗
train_set_1_merge = pd.concat([train_set_1,train_set_1_label],axis=1)#为了数据清洗方便，先将标签合并上去
train_set_1_merge = data_clean(train_set_1_merge)#清除不在范围的样本及标签
train_set_1 = train_set_1_merge.drop('label',axis=1)#将特征矩阵分离出来
train_set_1_label = train_set_1_merge['label']#将标签分离出来
#对测试集清洗
test_set_1_merge = pd.concat([test_set_1,test_set_1_label],axis=1)#为了数据清洗方便，先将标签合并上去
test_set_1_merge = data_clean(test_set_1_merge)#清除不在范围的样本及标签
test_set_1 = test_set_1_merge.drop('label',axis=1)#将特征矩阵分离出来
test_set_1_label = test_set_1_merge['label']#将标签分离出来

In [98]:
#将数据转换成多维数组并进行标准化
#将数据全部转换为矩阵
train_set_1 = np.array(train_set_1)
test_set_1 = np.array(test_set_1)
train_set_1_label = np.array(train_set_1_label)
test_set_1_label = np.array(test_set_1_label)
#对训练集及测试集的特征矩阵进行归一化
X_train = normalization(train_set_1)
X_test = normalization(test_set_1)
y_train = train_set_1_label
y_test = test_set_1_label

In [99]:
LR = LogisticRegression()#将scikit-learn包中的LogisticRegression模块重命名为LR

In [100]:
LR.fit(X_train,y_train)#运用训练集训练模型，参数全部使用默认值

LogisticRegression()

In [101]:
print('训练集错误率为：',1 - LR.score(X_train,y_train))#计算模型运用在训练集上的错误率

训练集错误率为： 0.15899581589958156


In [102]:
cross_val_score(LR, X_train, y_train, cv=10)#计算k折交叉验证的正确率

array([0.875     , 0.75      , 0.75      , 0.75      , 0.625     ,
       0.83333333, 0.875     , 0.79166667, 0.91666667, 0.7826087 ])

In [103]:
print('测试集错误率为：',1 - LR.score(X_test,y_test))#计算模型运用在测试集上的错误率

测试集错误率为： 0.2033898305084746
